It's a real pain to find historical stock prices these days. Both Google Finance and Yahoo Finance have decommissioned their APIs. The following is a dumb approach via the NASDAQ website. 

Go to https://www.nasdaq.com/market-activity/stocks/screener, enter a stock symbol, go to 'Historical Quotes', select 'YTD', then download the CSV file to a local directory. Prefix the filename with the stock symbol, e.g. "AAPL_HistoricalData_1627476057029". Do this for every stock of interest.

Then, you're ready to run the following code to load the data into Beneath.

In [1]:
import beneath
import pandas as pd
import numpy as np
import glob
from datetime import datetime

In [2]:
all_files = glob.glob("data/*.csv")
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['symbol'] = filename.split('/')[1].split('_')[0]
    li.append(df)

prices = pd.concat(li, axis=0, ignore_index=True)
prices = prices.rename(columns={'Date': 'date', 'Close/Last': 'close'})
prices['date'] = prices['date'].apply(lambda s: datetime.strptime(s, "%m/%d/%Y"))
prices = prices[['symbol', 'date', 'close']]

In [3]:
await beneath.write_full(
    "examples/wallstreetbets-analytics/stock_prices_daily",
    prices,
    key=["symbol", "date"],
    description="Daily stock prices for select symbols. Pulled from the NASDAQ Stock Screener on July 28th, 2021.",
    recreate_on_schema_change=True,
)

INFO:beneath:Flushed 1987 records to 1 instances (1987 total during session)
